In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import glob

In [ ]:
def number_of_pictures(path):
    c = glob.glob(path+'/*.jpg')
    c = [a.split('/')[-1] for a in c]
    return len(c)

In [ ]:
def convert_to_onehot(Y,C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


In [ ]:
path = "/train1/"
vid = cv2.VideoCapture(0)
def take_picture():
    
    h = number_of_pictures(path+'0');
    f = number_of_pictures(path+'1');
    p = number_of_pictures(path+'2');
    n = number_of_pictures(path+'3');
    print(h)
    while(1):
        ret, img = vid.read()
        if ret:
            cv2.imshow('picture', img)
            key = cv2.waitKey(2)
            
            if key == 104:
                print(key)
                cv2.imwrite(path+'0/'+str(h)+'.jpg',img)
                h += 1
                
            elif key == ord('f'):
                cv2.imwrite(path+'1/'+str(f)+'.jpg',img)
                f += 1

            elif key == ord('p'):
                cv2.imwrite(path+'2/'+str(p)+'.jpg',img)
                p += 1

            elif key == ord('n'):
                cv2.imwrite(path+'3/'+str(n)+'.jpg',img)
                n += 1
            elif key == ord('c'):
                cv2.destroyAllWindows()
                vid.release()
            

In [ ]:
take_picture()

In [ ]:
vid.release()
cv2.destroyAllWindows()

In [ ]:

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(96, (11,11),strides = (4,4), input_shape = (227,227,3),kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides= (2,2)))


model.add(tf.keras.layers.Conv2D(256, (5,5),strides = (1,1), input_shape = (27,27,96),kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides= (2,2)))


model.add(tf.keras.layers.Conv2D(384, (3,3),strides = (1,1), input_shape = (13,13,256),kernel_initializer='glorot_uniform'))

model.add(tf.keras.layers.Conv2D(384, (3,3),strides = (1,1), input_shape = (13,13,384),kernel_initializer='glorot_uniform'))

model.add(tf.keras.layers.Conv2D(256, (3,3),strides = (1,1), input_shape = (13,13,384),kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides= (2,2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(9216, activation= 'relu'))
model.add(tf.keras.layers.Dense(4096, activation = 'relu'))
model.add(tf.keras.layers.Dense(4096, activation= 'relu'))

model.add(tf.keras.layers.Dense(4, activation= 'softmax'))
path = "/robotic_arm/"
checkpoint_path = path+'Checkpoints/cp.cpkt'
model.load_weights(checkpoint_path)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), metrics=['accuracy'])
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), metrics=['accuracy'])

In [ ]:
checkpoint_path = path+'Checkpoints/cp.cpkt'
logdir = path+'Tensoboard_data'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True,verbose=1)
cp_logdir = tf.keras.callbacks.TensorBoard(log_dir = logdir,histogram_freq = 1000)



In [ ]:
def make_dataset(path):  
    images = []
    labels = []
    for i in range(4):
        data_path = glob.glob(path+str(i)+"/*.jpg")
        for j in data_path:
            image = cv2.imread(j,cv2.IMREAD_COLOR)
            image = cv2.resize(image, (227,227), interpolation = cv2.INTER_LANCZOS4)
            image = np.array(image)
            image = image.astype('int')
            images.append(image)
            labels.append(i)
            
    np.save("Robot_test.npy", images)
    np.save("Robot_test_label.npy",labels)
            

In [ ]:
path = "/train1/"
make_dataset(path)

In [ ]:
c = np.load("Robot_test[0].npy")
print(c)

In [ ]:
epoch = 10
X_train = np.load(path+"RoboticData.npy")
Y_train = np.load(path +"RoboticData_labels.npy")
Y_train = Y_train.astype('int')
Y_train = tf.one_hot(Y_train,4)
model.fit(X_train,Y_train,batch_size=64,epochs=epoch,validation_data = (X_train[0:1000],Y_train[0:1000]) , callbacks = [cp_callback])

In [ ]:
def predict(model, X_test):
    logits = model.predict(X_test)
    return np.argmax(logits, axis =1)

In [ ]:
path = "/robotic_arm/"
X_train = np.load("/Robot_test.npy")
Y_train = np.load("/Robot_test_label.npy")
Y_train = Y_train.astype('int')
print(Y_train.shape)
Y_train = convert_to_onehot(Y_train, 4)
print(Y_train.shape)

In [ ]:
# np.random.seed(0)
# np.random.shuffle(X_train)
# np.random.seed(0)
# np.random.shuffle(Y_train)


In [ ]:
while True:
    frame =cv2.VideoCapture(0)
    ret, img = frame.read()

    if ret:
        img = cv2.resize(img, (227,227), interpolation=cv2.INTER_CUBIC)
        cv2.imshow('Main', img)
        # adhere to TS graph input structure
#         numpy_frame = np.asarray(frame)
#         numpy_frame = cv2.normalize(numpy_frame.astype('int'), None, -0.5, .5, cv2.NORM_MINMAX)
        img = img.astype('float16')
        img = np.expand_dims(img, axis=0)

        print(predict(model ,img))

        

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()
            

In [ ]:
model.evaluate(X_train, Y_train, batch_size=8)